# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4934, uptime 0:00:22
mini_beamline                    RUNNING   pid 4935, uptime 0:00:22
random_walk                      RUNNING   pid 4936, uptime 0:00:22
random_walk_horiz                RUNNING   pid 4937, uptime 0:00:22
random_walk_vert                 RUNNING   pid 4938, uptime 0:00:22
simple                           RUNNING   pid 4939, uptime 0:00:22
thermo_sim                       RUNNING   pid 4940, uptime 0:00:22
trigger_with_pc                  RUNNING   pid 4941, uptime 0:00:22


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-18 16:51:38
Persistent Unique Scan ID: '8e285ffb-7be8-4ee8-a5b7-037c2276ce07'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 16:51:38.8 |     -1.000 |      97337 |
|         2 | 16:51:38.8 |     -0.500 |      98552 |
|         3 | 16:51:38.9 |      0.000 |      99267 |
|         4 | 16:51:38.9 |      0.500 |      97968 |
|         5 | 16:51:38.9 |      1.000 |      96672 |


+-----------+------------+------------+------------+
generator scan ['8e285ffb'] (scan num: 1)





('8e285ffb-7be8-4ee8-a5b7-037c2276ce07',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/8e285ffb-7be8-4ee8-a5b7-037c2276ce07-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-18 16:51:39
Persistent Unique Scan ID: '2ff593f9-a327-40d9-a00f-549544d25a90'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 16:51:39.4 |          -1 | 1063912481 |


|         2 | 16:51:39.6 |          -0 | 1058585850 |


|         3 | 16:51:39.7 |           0 | 1054129149 |


|         4 | 16:51:39.8 |           0 | 1053287677 |


|         5 | 16:51:39.9 |           1 | 1053794279 |
+-----------+------------+-------------+------------+
generator scan ['2ff593f9'] (scan num: 2)


('2ff593f9-a327-40d9-a00f-549544d25a90',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 16:51:40
Persistent Unique Scan ID: '84b785ee-4052-4f8b-95f1-92b95a7c6bd2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 16:51:40.4 | 1064988105 |


|         2 | 16:51:41.4 | 1123000871 |


|         3 | 16:51:42.4 | 1116951166 |


...

|        28 | 16:52:07.4 | 1059825201 |


|        29 | 16:52:08.4 | 1071006237 |


|        30 | 16:52:09.4 | 1121794175 |


+-----------+------------+------------+
generator count ['84b785ee'] (scan num: 3)





('84b785ee-4052-4f8b-95f1-92b95a7c6bd2',)